# 3 - API, Crawler, JSON e HTML

## Vereadoras/Vereadores da cidade de São Paulo - API e JSON

Links:
- [CMSP - Dados disponibilizados em formato aberto](http://www.saopaulo.sp.leg.br/transparencia/dados-abertos/dados-disponibilizados-em-formato-aberto/)
- [SPLEGIS - Sistema do Processo Legislativo](http://splegisws.camara.sp.gov.br/ws/ws2.asmx)
- [SPLEGIS - Lista de vereadores](http://splegisws.camara.sp.gov.br/ws/ws2.asmx?op=VereadoresCMSPJSON) 

In [1]:
import requests
import pandas as pd

In [2]:
url_api = 'http://splegisws.camara.sp.gov.br/ws/ws2.asmx/VereadoresCMSPJSON'

In [3]:
resultados = requests.get(url_api)
resultados.status_code

200

In [4]:
resultados.content

b'[{"chave":83,"nome":"ALBERTINO NOBRE","cargos":[{"chave":"V","nome":"Vice-presidente","inicio":"\\/Date(705283200000)\\/","fim":"\\/Date(725760000000)\\/","ente":{"chave":59,"nome":"Comiss\xc3\xa3o - ADMINISTRA\xc3\x87\xc3\x83O P\xc3\x9aBLICA"}},{"chave":"V","nome":"Vice-presidente","inicio":"\\/Date(697939200000)\\/","fim":"\\/Date(702518400000)\\/","ente":{"chave":59,"nome":"Comiss\xc3\xa3o - ADMINISTRA\xc3\x87\xc3\x83O P\xc3\x9aBLICA"}},{"chave":"V","nome":"Vice-presidente","inicio":"\\/Date(681004800000)\\/","fim":"\\/Date(697852800000)\\/","ente":{"chave":61,"nome":"Comiss\xc3\xa3o - EDUCACAO, CULTURA E ESPORTES"}},{"chave":"5","nome":"2\xc2\xba Secret\xc3\xa1rio","inicio":"\\/Date(694224000000)\\/","fim":"\\/Date(725760000000)\\/","ente":{"chave":5,"nome":"MESA DA CAMARA MUNICIPAL DE SAO PAULO - 01/01/1992 a 31/12/1992"}}],"mandatos":[{"inicio":"\\/Date(599616000000)\\/","fim":"\\/Date(725760000000)\\/","partido":{"chave":54,"nome":"PARTIDO TRABALHISTA BRASILEIRO","sigla":"PTB"

### Igualzinho a um csv, né?

<img src="https://media.giphy.com/media/fpXxIjftmkk9y/giphy.gif" align="left">

In [5]:
type(resultados.content)

bytes

In [7]:
resultados_json = resultados.json()
resultados_json

[{'cargos': [{'chave': 'V',
    'ente': {'chave': 59, 'nome': 'Comissão - ADMINISTRAÇÃO PÚBLICA'},
    'fim': '/Date(725760000000)/',
    'inicio': '/Date(705283200000)/',
    'nome': 'Vice-presidente'},
   {'chave': 'V',
    'ente': {'chave': 59, 'nome': 'Comissão - ADMINISTRAÇÃO PÚBLICA'},
    'fim': '/Date(702518400000)/',
    'inicio': '/Date(697939200000)/',
    'nome': 'Vice-presidente'},
   {'chave': 'V',
    'ente': {'chave': 61, 'nome': 'Comissão - EDUCACAO, CULTURA E ESPORTES'},
    'fim': '/Date(697852800000)/',
    'inicio': '/Date(681004800000)/',
    'nome': 'Vice-presidente'},
   {'chave': '5',
    'ente': {'chave': 5,
     'nome': 'MESA DA CAMARA MUNICIPAL DE SAO PAULO - 01/01/1992 a 31/12/1992'},
    'fim': '/Date(725760000000)/',
    'inicio': '/Date(694224000000)/',
    'nome': '2º Secretário'}],
  'chave': 83,
  'filiacoes': [{'inicio': '/Date(410227200000)/',
    'partido': {'chave': 54,
     'nome': 'PARTIDO TRABALHISTA BRASILEIRO',
     'sigla': 'PTB'}}],
  'mand

In [8]:
type(resultados_json)

list

In [9]:
len(resultados_json)

307

In [10]:
resultados_json[0]

{'cargos': [{'chave': 'V',
   'ente': {'chave': 59, 'nome': 'Comissão - ADMINISTRAÇÃO PÚBLICA'},
   'fim': '/Date(725760000000)/',
   'inicio': '/Date(705283200000)/',
   'nome': 'Vice-presidente'},
  {'chave': 'V',
   'ente': {'chave': 59, 'nome': 'Comissão - ADMINISTRAÇÃO PÚBLICA'},
   'fim': '/Date(702518400000)/',
   'inicio': '/Date(697939200000)/',
   'nome': 'Vice-presidente'},
  {'chave': 'V',
   'ente': {'chave': 61, 'nome': 'Comissão - EDUCACAO, CULTURA E ESPORTES'},
   'fim': '/Date(697852800000)/',
   'inicio': '/Date(681004800000)/',
   'nome': 'Vice-presidente'},
  {'chave': '5',
   'ente': {'chave': 5,
    'nome': 'MESA DA CAMARA MUNICIPAL DE SAO PAULO - 01/01/1992 a 31/12/1992'},
   'fim': '/Date(725760000000)/',
   'inicio': '/Date(694224000000)/',
   'nome': '2º Secretário'}],
 'chave': 83,
 'filiacoes': [{'inicio': '/Date(410227200000)/',
   'partido': {'chave': 54,
    'nome': 'PARTIDO TRABALHISTA BRASILEIRO',
    'sigla': 'PTB'}}],
 'mandatos': [{'fim': '/Date(7257

In [11]:
type(resultados_json[0])

dict

In [12]:
lista_novos_dados = []

for item in resultados_json:
    novos_dados = {
        'nome': item.get('nome'),
        'qtd_mandatos': len(item.get('mandatos')),
        'qtd_cargos': len(item.get('cargos'))
    } 
    
    lista_novos_dados.append(novos_dados)

In [13]:
resultados_df = pd.DataFrame(lista_novos_dados)
resultados_df.head(5)

,nome,qtd_cargos,qtd_mandatos
0,ALBERTINO NOBRE,4,2
1,ALFREDO MARTINS,4,2
2,ALMIR GUIMARAES,6,3
3,ANTONIO CARLOS FERNANDES,0,1
4,ANDRADE FIGUEIRA,1,2


In [14]:
resultados_df.sort_values(['qtd_mandatos', 'qtd_cargos'], ascending=False).head(10)

,nome,qtd_cargos,qtd_mandatos
90,GILSON BARRETO,43,8
38,WADIH MUTRAN,28,8
60,ARSELINO TATTO,28,8
117,TONINHO PAIVA,37,7
121,AURÉLIO NOMURA,36,7
79,ROBERTO TRIPOLI,35,7
21,JOOJI HATO,27,7
138,DALTON SILVANO,62,6
143,MILTON LEITE,33,6
137,PAULO FRANGE,32,6


### Podemos salvar o que extraímos em csv?

<img src="https://media.giphy.com/media/hVYVYZZBgF50k/giphy.gif" align="left">

In [15]:
resultados_df.to_csv('lista_vereadores.csv', index=False)

## Vereadoras/Vereadores da cidade de São Paulo - Crawler e HTML


O [Scrapy](https://docs.scrapy.org/en/latest/index.html) é um framework para criação de aplicações que querem "crawlear" sites.

Biblioteca - É um conjunto de códigos que alguém ecapsulou em um único lugar. De forma que outras pessoas possam reutilizar o código.

Framework - Também é um conjunto de códigos encapsulados em algum lugar que possibilitam a reutilização mas permitem a extensão e adaptação desse código. Exemplos: Confecção de telas de login, mapeamento para banco de dados

Crawler ou Spider - Aplicação/script para navegar e retirar informações de página sites na Internet, e para isso pode utilizar de recursão.

API - Application Programming Interface - Conceito abstrato que significa uma espécie de interface com os dados ou funcionalidades de uma determinada aplicação.

HTML
http://www.saopaulo.sp.leg.br/vereadores/

## Scrapy

In [31]:
import scrapy
import json
import csv

In [34]:
teste = requests.get('http://quotes.toscrape.com/tag/humor/')
teste.status_code

200

In [35]:
teste.content

b'<!DOCTYPE html>\n<html lang="en">\n<head>\n\t<meta charset="UTF-8">\n\t<title>Quotes to Scrape</title>\n    <link rel="stylesheet" href="/static/bootstrap.min.css">\n    <link rel="stylesheet" href="/static/main.css">\n</head>\n<body>\n    <div class="container">\n        <div class="row header-box">\n            <div class="col-md-8">\n                <h1>\n                    <a href="/" style="text-decoration: none">Quotes to Scrape</a>\n                </h1>\n            </div>\n            <div class="col-md-4">\n                <p>\n                \n                    <a href="/login">Login</a>\n                \n                </p>\n            </div>\n        </div>\n    \n\n<h3>Viewing tag: <a href="/tag/humor/page/1/">humor</a></h3>\n\n<div class="row">\n    <div class="col-md-8">\n\n    <div class="quote" itemscope itemtype="http://schema.org/CreativeWork">\n        <span class="text" itemprop="text">\xe2\x80\x9cThe person, be it gentleman or lady, who has not pleasure 

In [38]:
teste.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

[Quotes to Scrape](http://quotes.toscrape.com/tag/humor/)

In [51]:
%%writefile quotes.py
import scrapy


class QuotesSpider(scrapy.Spider):
    name = 'quotes'
    
    def start_requests(self):
        urls = [
            'http://quotes.toscrape.com/tag/humor/page/1/',
            'http://quotes.toscrape.com/tag/humor/page/2/',
        ]
        
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        for quote in response.css('div.quote'):
            yield {
                'text': quote.css('span.text::text').get(),
                'author': quote.xpath('span/small/text()').get(),
            }

Overwriting lorena.py


In [52]:
!ls

1_intro.ipynb	      lorena.py		  __pycache__  vereadores
crawler_scrapy.ipynb  noticias_api.ipynb  quotes.json
lista_vereadores.csv  noticias_xml.ipynb  teste


In [53]:
!scrapy runspider quotes.py -o quotes.json

2019-11-21 19:45:40 [scrapy.utils.log] INFO: Scrapy 1.5.1 started (bot: scrapybot)
2019-11-21 19:45:40 [scrapy.utils.log] INFO: Versions: lxml 4.2.5.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.5.0, w3lib 1.21.0, Twisted 18.7.0, Python 3.5.6 |Anaconda, Inc.| (default, Aug 26 2018, 21:41:56) - [GCC 7.3.0], pyOpenSSL 18.0.0 (OpenSSL 1.0.2t  10 Sep 2019), cryptography 2.3.1, Platform Linux-5.0.0-23-generic-x86_64-with-debian-buster-sid
2019-11-21 19:45:40 [scrapy.crawler] INFO: Overridden settings: {'FEED_FORMAT': 'json', 'SPIDER_LOADER_WARN_ONLY': True, 'FEED_URI': 'quotes.json'}
2019-11-21 19:45:41 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2019-11-21 19:45:41 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware

In [98]:
%%writefile items.py
import scrapy

class Vereador(scrapy.Item):
    
    nome = scrapy.Field()
    email = scrapy.Field()
    rede_social = scrapy.Field()

Overwriting items.py


In [101]:
%%writefile vereadores_sp.py
import scrapy
from scrapy.loader import ItemLoader
from items import Vereador

class VereadoresSpider(scrapy.Spider):
    name = 'veradores_sp'
    start_urls = ['http://www.saopaulo.sp.leg.br/vereadores/']

    def parse(self, response):      
        lista_veradores = response.css('p.vereador-name a::attr(href)').getall()
        
        for item in lista_veradores:
            yield response.follow(item, self.parse_vereador)
    
    
    def parse_vereador(self, response):
        
        item_loader = ItemLoader(Vereador(), response)
        
        item_loader.add_css('nome', 'p.vereador-name a::text')
        item_loader.add_css('email', 'div.vereador-data ul li a::text')
        item_loader.add_xpath('rede_social', '/html/body/div[2]/div/div[3]/div/div/article/header/div[2]/ul/li[5]/a/@href')

    
        return item_loader.load_item()

Overwriting vereadores_sp.py


In [69]:
!ls

1_intro.ipynb	      lorena.py		  __pycache__  vereadores
crawler_scrapy.ipynb  noticias_api.ipynb  quotes.json  vereadores_sp.py
lista_vereadores.csv  noticias_xml.ipynb  teste


In [103]:
!scrapy runspider vereadores_sp.py -o vereadores.json

2019-11-21 21:56:38 [scrapy.utils.log] INFO: Scrapy 1.5.1 started (bot: scrapybot)
2019-11-21 21:56:38 [scrapy.utils.log] INFO: Versions: lxml 4.2.5.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.5.0, w3lib 1.21.0, Twisted 18.7.0, Python 3.5.6 |Anaconda, Inc.| (default, Aug 26 2018, 21:41:56) - [GCC 7.3.0], pyOpenSSL 18.0.0 (OpenSSL 1.0.2t  10 Sep 2019), cryptography 2.3.1, Platform Linux-5.0.0-23-generic-x86_64-with-debian-buster-sid
2019-11-21 21:56:38 [scrapy.crawler] INFO: Overridden settings: {'SPIDER_LOADER_WARN_ONLY': True, 'FEED_URI': 'vereadores.json', 'FEED_FORMAT': 'json'}
2019-11-21 21:56:38 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2019-11-21 21:56:38 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddle

2019-11-21 21:56:43 [scrapy.core.scraper] DEBUG: Scraped from <200 http://www.saopaulo.sp.leg.br/vereador/ze-turin/>
{'email': ['zeturin@saopaulo.sp.leg.br',
           'http://www.cidadaoparticipa.com.br/zeturin',
           'https://www.facebook.com/ZeTurinVereador/',
           'https://www.instagram.com/zeturin_vereador'],
 'nome': ['Zé Turin'],
 'rede_social': ['https://www.facebook.com/ZeTurinVereador/']}
2019-11-21 21:56:43 [scrapy.core.scraper] DEBUG: Scraped from <200 http://www.saopaulo.sp.leg.br/vereador/milton-leite/>
{'email': ['miltonleite@saopaulo.sp.leg.br'], 'nome': ['Milton Leite']}
2019-11-21 21:56:44 [scrapy.core.scraper] DEBUG: Scraped from <200 http://www.saopaulo.sp.leg.br/vereador/dr-milton-ferreira/>
{'email': ['drmiltonferreira@saopaulo.sp.leg.br'], 'nome': ['Milton Ferreira']}
2019-11-21 21:56:44 [scrapy.core.scraper] DEBUG: Scraped from <200 http://www.saopaulo.sp.leg.br/vereador/patricia-bezerra/>
{'email': ['vereadora@patriciabezerra.com.br',
           'h

2019-11-21 21:56:48 [scrapy.core.scraper] DEBUG: Scraped from <200 http://www.saopaulo.sp.leg.br/vereador/natalini/>
{'email': ['natalini@saopaulo.sp.leg.br',
           'http://www.natalini.com.br',
           'https://www.facebook.com/gilberto.natalini',
           'https://www.instagram.com/gilbertonatalinisp/?hl=pt',
           'https://twitter.com/gnatalini'],
 'nome': ['Gilberto Natalini'],
 'rede_social': ['https://www.facebook.com/gilberto.natalini']}
2019-11-21 21:56:49 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://www.saopaulo.sp.leg.br/vereador/george-hato/> (referer: http://www.saopaulo.sp.leg.br/vereadores/)
2019-11-21 21:56:49 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://www.saopaulo.sp.leg.br/vereador/gilberto-nascimento/> (referer: http://www.saopaulo.sp.leg.br/vereadores/)
2019-11-21 21:56:49 [scrapy.core.scraper] DEBUG: Scraped from <200 http://www.saopaulo.sp.leg.br/vereador/george-hato/>
{'email': ['gvhato@saopaulo.sp.leg.br'], 'nome': ['George Hato

2019-11-21 21:56:52 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://www.saopaulo.sp.leg.br/vereador/caio-miranda-carneiro/> (referer: http://www.saopaulo.sp.leg.br/vereadores/)
2019-11-21 21:56:52 [scrapy.core.scraper] DEBUG: Scraped from <200 http://www.saopaulo.sp.leg.br/vereador/caio-miranda-carneiro/>
{'email': ['caiomirandacarneiro@saopaulo.sp.leg.br',
           'http://www.facebook.com/caiomirandacarneiro',
           'http://www.instagram.com/caiomirandacarneiro',
           'http://www.twitter.com/caiomcarneiro'],
 'nome': ['Caio Miranda Carneiro'],
 'rede_social': ['http://www.instagram.com/caiomirandacarneiro']}
2019-11-21 21:56:52 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://www.saopaulo.sp.leg.br/vereador/atilio-francisco/> (referer: http://www.saopaulo.sp.leg.br/vereadores/)
2019-11-21 21:56:52 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://www.saopaulo.sp.leg.br/vereador/arselino-tatto/> (referer: http://www.saopaulo.sp.leg.br/vereadores/)
2019-11-2

In [104]:
!ls

1_intro.ipynb	      lorena.py		  quotes.json	   vereadores_sp.py
crawler_scrapy.ipynb  noticias_api.ipynb  teste
items.py	      noticias_xml.ipynb  vereadores
lista_vereadores.csv  __pycache__	  vereadores.json


### Arquitetura do Scrapy

Todo: Explicar herança de Spider e Itens por exemplo

[Tutorial do próprio Scrapy](https://docs.scrapy.org/en/latest/intro/tutorial.html)

Após rodar o comando `scrapy startproject seu_projeto`

    seu_projeto/
        scrapy.cfg            # Arquivo de configurações para o deploy do projeto

        tutorial/             # Módulo python do projeto
            __init__.py

            items.py          # Arquivo com as definições do *item* - dado que você está buscando

            middlewares.py    # Arquivo

            pipelines.py      # Arquivo

            settings.py       # Arquivo de configurações do projeto

            spiders/          # Módulo com os spiders
                __init__.py

#### Ah massa! Fizemos tudo. Temos um forma automatizada de extrair, analisar e salvar esse dado... O que acontece agora?

# A estrutura HTML vai mudar!

<img src="https://media.tenor.com/images/75457a53b6ad99b2c857353eb4e21f76/tenor.gif" align="left">